<a href="https://colab.research.google.com/github/elegant/dl/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.1 MB/s eta 0:00:00


In [2]:
import gensim
import pandas as pd

### Reading dataset
Subset of Amazon reviews from the Cell Phones & Accessories category (.json file read using pandas)

In [9]:
import urllib.request
from zipfile import ZipFile
import os


#What to download-
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz"
filename = "reviews_Cell_Phones_and_Accessories_5.json.gz"
urllib.request.urlretrieve(url, filename)

print("Unzipping")
os.system("gunzip reviews_Cell_Phones_and_Accessories_5.json.gz")

Unzipping


512

In [11]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [12]:
df.shape

(194439, 9)

### Preprocessing

In [18]:
df["reviewText"].loc[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [17]:
review_text = df["reviewText"].apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [15]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

### Training Word2Vec model

window of size 10 (10 words before the present word and 10 words ahead)
A sentence with at least 2 words should only be considered.
Workers define how many CPU threads to be used.

In [ ]:
#Initializing model

model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)


In [ ]:
#Building vocab

model.build_vocab(review_text, progress_per=1000)
model.corpus_count

194439

In [ ]:
#Training the model
print(model.epochs)
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

5

### Saving the model

In [30]:
model.save("./word2vec.model")

### App
#### Finding similar words, similarity between words

In [31]:
model.wv.most_similar("bad")

[('terrible', 0.6625651121139526),
 ('shabby', 0.6597665548324585),
 ('horrible', 0.5999440550804138),
 ('good', 0.5742393732070923),
 ('awful', 0.5414789915084839),
 ('poor', 0.5269405841827393),
 ('disappointing', 0.5263199210166931),
 ('cheap', 0.5205562114715576),
 ('okay', 0.5158125162124634),
 ('crappy', 0.5121426582336426)]

In [32]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.5178923

In [33]:
model.wv.similarity(w1="great", w2="good")

0.7914358